<center><span style="background-color: rgb(251, 160, 38); font-size: 32px;">  <b>NUCLIO DIGITAL SCHOOL -</b> MASTER EN DATA SCIENCE  </span></center>
    
<br>
    
<center><a href = https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png > <img src="https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png" width=400 height=100><a/></center>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">
    <center> <span style="font-size: 26px;"> PCA and Collaborative Filtering </span> </center>

+ Session: **PCA and Collaborative Filtering**
+ Module: **Unsupervised Learning**
+ Course: **Data Science Master 0921**
+ Professor: **Christa Santos**

<br>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

<a id = "toc"></a>
# Table of Contents
[1. Objective](#objectives)

[2. Import the main libraries](#import_modules)

[3. Import the data](#import_data)

[4. Exploratory Data Analysis (EDA)](#eda)

---> [EDA anime df](#df1)

---> [EDA ratings df](#df2)

[5. Join final animes with user ratings](#join)

[6. Dimensionality Reduction with PCA](#pca)

[7. Calculate similarity between users and animes reviews](#collaborative_filtering)

[8. User based Recommendation](#recommendation_users)

[9. Product based Recommendation](#recommendation_animes)

[10. Conclusion](#conclusion)

<a id = "objectives"></a>
## 1. Objective
[Table of Contents](#toc)

One of the areas of machine learning with which we interact almost daily are the Collaborative Filtering models. In today's notebook, we will create 2 CF models: **one based on users (people similar to you, have bought / seen / liked these things) and another based on products (people who have bought this product, have also bought these).**

The dataset that we are going to use is a Japanese Animes dataset and it can be downloaded at the following [link](https://www.kaggle.com/CooperUnion/anime-recommendations-database)

Our main objectives will be:
1. **Do an initial exploration of the two datasets** and understand the distribution of the data.

2. **Extract some useful variables such as: anime genre** and eliminate users who have not rated the anime.

3. **Reduce the dimensionality of our DataFrame using the PCA**

4. **Segment our clients using the reduced dataset**

5. **Use the similarity of the cosine to make recommendations to our customers (user and product based)**

<a id = "import_modules"></a>
## 2. Import the main libraries
[Table of Contents](#toc)

In this section of the kernel we are going to load the main libraries that we are going to use in our notebook.

In [ ]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# operating system
import os

# time calculation to track some processes
import time

# numeric and matrix operations
import numpy as np
import pandas as pd

# scientific computations library
import scipy as sp

# loading ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

# import the function to compute cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

PATH_FOLDER = os.path.join(os.getcwd(), "data\\anime_datasets")

In [ ]:
PATH_FOLDER

<a id = "import_data"></a>
## 3. Import the data
[Table of Contents](#toc)

In this section of the kernel we are going to load the main datasets that we are going to use to build our recommender.

In [ ]:
PATH_ANIME = os.path.join(PATH_FOLDER, 'anime.csv')

anime_df = pd.read_csv(PATH_ANIME)

In [ ]:
PATH_RATING = os.path.join(PATH_FOLDER, 'rating.csv')

rating_df = pd.read_csv(PATH_RATING)

In [ ]:
VALUE_TO_IMPUTE = -1

<a id = "eda"></a>
## 4. Exploratory Data Analysis (EDA)
[Table of Contents](#toc)

In the EDA section we will make **a first approximation to our data** to see its composition and what variables we have at our disposal.

<a id = "df1"></a>
### 4.1 Anime dataset
[Table of Contents](#toc)

Quick EDA on **anime dataset**

In [ ]:
def report_df(df, verbose = True):
    '''
    Makes a simple report on the supplied DataFrame.
    '''
    print(df.info(verbose = verbose))
    total_nulos = df.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")

In [ ]:
report_df(anime_df)

anime_df.head()

We note that we have some nulls and we will have to deal with them.

In [ ]:
anime_df.isnull().sum()

We don't have duplicate anime_id's, as you would expect.

In [ ]:
sum(anime_df["anime_id"].value_counts() > 1)

By type of anime, we see that the most popular is the one on TV.

In [ ]:
anime_df["type"].value_counts().plot(kind = "bar", title = "Animes by type");

In our report_df we have seen that ** episodes ** seemed to be numeric, but it could contain other types of data (because it is object), we convert this column to a number.

In [ ]:
anime_df["episodes"] = pd.to_numeric(anime_df["episodes"], errors = "coerce")
anime_df["episodes"].fillna(1, inplace = True)

In [ ]:
anime_df.max()

In the next section we are going to analyze the distribution of the animes based on the number of episodes they have.

In [ ]:
count_per_episodes = anime_df["episodes"].value_counts().to_frame().reset_index()
count_per_episodes.columns = ["nr_episodes", "nr_films"]
count_per_episodes.head()

In [ ]:
count_per_episodes.sort_values("nr_episodes", ascending = True, inplace = True)
count_per_episodes["pct_over_total"] = count_per_episodes["nr_films"]/count_per_episodes["nr_films"].sum()

In [ ]:
count_per_episodes.head()

Almost half of the anime is a single episode (48.94%)

In [ ]:
NR = 30

# instanciate the figure
fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot(111)

# separete the data
x = count_per_episodes["nr_episodes"].values[:NR]
y = count_per_episodes["nr_films"].values[:NR]
y_pct = count_per_episodes["pct_over_total"].values[:NR]

# plot the data
barplot = ax.bar(x, y)

# add text to each column
for rect, y_pct_ in zip(barplot, y_pct):
    y_pct_ = round(y_pct_*100, 2)
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, f"{height}:{y_pct_}%", ha = 'center', va = "bottom", rotation = 60)

# change the xticks
ax.set_xticks(np.arange(0, NR + 1))

# add title
total_y_pct = round(sum(y_pct)*100, 2)
ax.set_title(f"Distribution of the first {NR} animes ({total_y_pct}% of the total)");

We are going to carry out an analysis similar to the previous one, but now we will see how the animes are distributed based on the average score.

To have only 10 groups, we are first going to round the mean score.

In [ ]:
anime_df["rating"].fillna(np.mean(anime_df["rating"]), inplace = True)
anime_df["ceil_rating"] = anime_df["rating"].apply(lambda rating: np.round(rating, 0))

In [ ]:
anime_df[["ceil_rating","rating"]].groupby(["ceil_rating","rating"]).size()

The most common score is a 7 and it is found in 4,579 animes (37.25% of the total).

In [ ]:
count_per_rating = anime_df["ceil_rating"].value_counts().to_frame()\
.reset_index().sort_values("index", ascending = True)

# instanciate the figure
fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot(111)

# separete the data
x = count_per_rating["index"].values
y = count_per_rating["ceil_rating"].values
y_pct = y/sum(y)

# plot the data
barplot = ax.bar(x, y)

# add text to each column
for rect, y_pct_ in zip(barplot, y_pct):
    y_pct_ = round(y_pct_*100, 2)
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, f"{height} - {y_pct_}%", ha = 'center', va = "bottom")
    
ax.set_xticks(np.arange(0, 11))
ax.set_title("Distribution of the number of animes by rating")

In this section, we will extract the genres of each anime and convert it into columns, which we can add to the initial dataset.

In [ ]:
anime_df["genre"].fillna("Unknown", inplace = True)

In [ ]:
ll = anime_df["genre"].str.split(",").to_list()

In [ ]:
anime_df.head()

In [ ]:
ll_with_id = []

for ids, lists in zip(list(anime_df["anime_id"].values), ll):
    list_ = [ids]
    for values in lists:
        list_.append(values)
    ll_with_id.append(list_)

In [ ]:
anime_by_genre = pd.DataFrame(ll_with_id)

In [ ]:
anime_by_genre = pd.DataFrame(ll_with_id).melt(id_vars = 0)
anime_by_genre["value"].fillna("Unknown", inplace = True)
anime_by_genre.rename(columns = {0:"anime_id"}, inplace = True)
anime_by_genre.head()

In [ ]:
anime_by_genre = anime_by_genre.pivot_table(index = "anime_id", columns = "value", aggfunc = len, fill_value = 0)
anime_by_genre.head()

In [ ]:
new_columns = [tupple[1] for tupple in list(anime_by_genre.columns)]
new_columns = list(map(lambda text: text[1:] if text[0] == " " else text, new_columns))

In [ ]:
anime_by_genre.columns = new_columns
anime_by_genre.reset_index(inplace = True)
anime_by_genre.head()

In [ ]:
anime_df.shape[0] == anime_by_genre.shape[0]

In [ ]:
# anime_df = pd.merge(anime_df, anime_by_genre, how = "left", on = ["anime_id"])

In [ ]:
anime_df.head()

<a id = "df2"></a>
### 4.2 Ratings dataset
[Table of Contents](#toc)

Quick EDA on **ratings dataset**

In [ ]:
report_df(rating_df)

rating_df.head()

In [ ]:
rating_gb = rating_df["rating"].value_counts().reset_index().sort_values("index", ascending = True)
rating_gb.sort_values("index", ascending = True, inplace = True)

fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot()

x = rating_gb["index"]
y = rating_gb["rating"]
y_pct = y/sum(y)

barplot = ax.bar(x, y)

# add text to each column
for rect, y_pct_ in zip(barplot, y_pct):
    y_pct_ = round(y_pct_*100, 2)
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, f"{height} - {y_pct_}%", ha = 'center', va = "bottom", rotation = 60)

ax.set_xticks(x);

We observe that we have **almost 20% of animes without reviews.**

We are going to see the users with the most reviews.

In [ ]:
user_pivot = rating_df["user_id"].value_counts()
user_pivot.head()

In [ ]:
# User 48766 is clearly a "weird" user. He has more than 10k reviews, but they are all null.
rating_df[rating_df["user_id"] == 48766].shape[0] == -rating_df[rating_df["user_id"] == 48766]["rating"].sum()

In [ ]:
print(rating_df[rating_df["user_id"] == 48766].shape[0])
print(rating_df[rating_df["user_id"] == 48766]["rating"].sum())

We are going to eliminate from our DataFrame all those users who have scored all -1

In [ ]:
s = rating_df.groupby('user_id')['rating'].apply(set)

In [ ]:
s

In [ ]:
user_id_no_reviews = s.to_frame()[s.to_frame()["rating"] == {-1}].index

In [ ]:
user_id_no_reviews

In [ ]:
rating_df = rating_df[-rating_df["user_id"].isin(user_id_no_reviews)]

In [ ]:
rating_df[rating_df["user_id"] == 2]["rating"].value_counts()

<a id = "join"></a>
## 5. Join final animes with user ratings
[Table of Contents](#toc)

Once we have analyzed our DataFrames, we are going to do a join **by anime_id.**

In [ ]:
rating_df.rename(columns = {"rating":"user_rating"}, inplace = True)
anime_df.rename(columns = {"rating":"average_rating"}, inplace = True)
df_final = pd.merge(rating_df, anime_df, on = "anime_id")

In [ ]:
df_final.head()

We filter only by type == TV to have a more manageable dataset.

In [ ]:
df_final["user_rating"].replace(-1, np.nan, inplace = True)

In [ ]:
df_final = df_final[df_final["type"] == "TV"]

In [ ]:
df_final = df_final.pivot_table(index = "user_id", columns = "name", values = "user_rating")

In [ ]:
df_final.head()

In [ ]:
df_final.isnull().sum()

In [ ]:
VALUE_TO_IMPUTE

In [ ]:
if VALUE_TO_IMPUTE != "MEAN":
    df_final.fillna(VALUE_TO_IMPUTE, inplace = True)

elif VALUE_TO_IMPUTE == "MEAN":
    imputer = SimpleImputer(strategy = "mean")
    X_imputed = imputer.fit_transform(df_final)
    df_final = pd.DataFrame(X_imputed, index = df_final.index, columns = df_final.columns)

In [ ]:
df_final.head()

In [ ]:
df_final.shape

<a id = "pca"></a>
## 6. Dimensionality Reduction with PCA
[Table of Contents](#toc)

The PCA is the most widely used method of dimensionality reduction and with a long theoretical development behind it (it was [invented in 1901] (https://en.wikipedia.org/wiki/Principal_component_analysis) by Karl Pearson.

The basic idea behind the PCA is the projection of a dataset with **n** dimensions to another **m** dimensions, where **n > m**, in such a way that the dataset in the new space preserves the maximum amount of information (variance) of the original space. Therefore, the new reduced dataset is a linear combination of the original **n** dimensions.

When we initialize the sklearn PCA, we have to specify the number of components that we want our new dataset to have.

In [ ]:
st = time.time()
pca = PCA(n_components = 30)
pca.fit(df_final)
pca_samples = pca.transform(df_final)
et = time.time()
print("Total PCA took {} minutes".format(round((et - st)/60, 2)))

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative explained variance');

In [ ]:
pca_df = pd.DataFrame(pca_samples[:, 0:20])

In [ ]:
pca_df

As you might expect, the projected dataset "loses" information when performing the transformation.

Another way to initialize the PCA, is instead of specifying the number of components, tell the PCA how much minimum variance the original dataset should contain (it has to be a number between 0 - 1).

pca = PCA (n_components = 0.3)

In [ ]:
pca = PCA(n_components = 0.7)
pca

<a id = "collaborative_filtering"></a>
## 7. Calculate similarity between users and animes reviews
[Table of Contents](#toc)

In this section we are going to calculate the cosine similarity for 4000 customers (and the products they have evaluated).

In [ ]:
CALCULATE_SIMILARITY = False

In [ ]:
if CALCULATE_SIMILARITY:

    st = time.time()
    df_final_norm = df_final.apply(lambda x: (x - np.mean(x))/(np.max(x) - np.min(x)), axis = 1)
    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    df_final_norm = df_final_norm[df_final_norm.index < 3000]
    sparse_ratings = sp.sparse.csr_matrix(df_final_norm.values)

    st = time.time()
    
    item_similarity = cosine_similarity(sparse_ratings.T)
    item_sim_df = pd.DataFrame(item_similarity, index = df_final_norm.columns, columns = df_final_norm.columns)
    item_sim_df.to_pickle(os.path.join(PATH_FOLDER, "item_similarity.pkl"))

    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = df_final_norm.index, columns = df_final_norm.index)
    user_sim_df.to_pickle(os.path.join(PATH_FOLDER, "user_similarity.pkl"))
    
    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))
    
else:
    user_sim_df = pd.read_pickle(os.path.join(PATH_FOLDER, "user_similarity.pkl"))
    item_sim_df = pd.read_pickle(os.path.join(PATH_FOLDER, "item_similarity.pkl"))

In [ ]:
user_sim_df.head()

In [ ]:
item_sim_df.head()

<a id = "recommendation_users"></a>
## 8. User based Recommendation
[Table of Contents](#toc)

In [ ]:
def top_users(user, df):
    '''
    This function prints the top 10 similar users based on cosine similarity.
    '''
    
    if user not in df.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    
    sim_users = df.sort_values(by = user, ascending=False).index[1:11]
    sim_values = df.sort_values(by = user, ascending=False).loc[:,user].tolist()[1:11]
    
    for user, sim in zip(sim_users, sim_values):
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 

In [ ]:
def compare_2_users(user1, user2, df, nr_animes):
    '''
    Returns a DataFrame with top 10 animes by 2 similar users (based on cosine similarity).
    '''

    top_10_user_1 = df[df.index == user1].melt().sort_values("value", ascending = False)[:nr_animes]
    top_10_user_1.columns = ["name_user_{}".format(user1), "rating_user_{}".format(user1)]
    top_10_user_1 = top_10_user_1.reset_index(drop = True)

    top_10_user_2 = df[df.index == user2].melt().sort_values("value", ascending = False)[:nr_animes]
    top_10_user_2.columns = ["name_user_{}".format(user2), "rating_user_{}".format(user2)]
    top_10_user_2 = top_10_user_2.reset_index(drop = True)

    combined_2_users = pd.concat([top_10_user_1, top_10_user_2], axis = 1, join = "inner")
    
    return combined_2_users

In [ ]:
user1 = 20

user2 = 2390

top_users(user1, user_sim_df)

combined_2_users = compare_2_users(user1, user2, df_final, 10)
combined_2_users

<a id = "recommendation_animes"></a>
## 9. Product based Recommendation
[Table of Contents](#toc)

In [ ]:
def top_animes(name, df):
    '''
    This functions prints top 10 similar animes, based on the reviews of the users.
    '''
    print('Similar shows to {} include:\n'.format(name))
    
    index = item_sim_df[name].sort_values(ascending = False).index[1:11]
    values = item_sim_df[name].sort_values(ascending = False).values[1:11]

    for i, (index_, values_) in enumerate(zip(index, values)):
        print('No. {}: {} ({})'.format(i + 1, index_, round(values_, 3)))

In [ ]:
top_animes("07-Ghost", item_sim_df)

<a id = "conclusion"></a>
## 10. Conclusion
[Table of Contents](#toc)

In this Notebook we have explored some of the most common techniques used in unsupervised learning such as: **PCA, KMeans.**

Subsequently, we have used the **"cosine similarity"** metric to create two collaborative filtering models: **user and product based.**

We have seen how UL techniques are very useful and can be used in countless fields from: **data visualization, creation of new variables (the main components), reducing dimensionality to speed up learning, among others.**